En este endpoint debo devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [35]:
import pandas as pd
import pyarrow.parquet as pq

In [36]:
df_steam_games = pq.read_table('Datos/df_steam_games_limpio_final.parquet').to_pandas()

In [37]:
df_items = pq.read_table('Datos/df_items_limpio_compr.parquet').to_pandas()

In [38]:
df_steam_games.head()

,genres,app_name,tags,specs,price,id,developer,anio
0,action,lost summoner kitty,strategy,single player,4.99,761140,kotoshiro,2018
1,action,lost summoner kitty,action,single player,4.99,761140,kotoshiro,2018
2,action,lost summoner kitty,indie,single player,4.99,761140,kotoshiro,2018
3,action,lost summoner kitty,casual,single player,4.99,761140,kotoshiro,2018
4,action,lost summoner kitty,simulation,single player,4.99,761140,kotoshiro,2018


In [6]:
df_items.head()

,user_id,items_count,item_id,item_name,playtime_forever
0,76561197970982479,277,10,counter strike,6.0
1,76561197970982479,277,20,team fortress classic,0.0
2,76561197970982479,277,30,day of defeat,7.0
3,76561197970982479,277,40,deathmatch classic,0.0
4,76561197970982479,277,50,half life: opposing force,0.0


Selecciono las columnas relevantes para el análisis

In [39]:
df_steam_games = df_steam_games[['genres','id','anio']]
df_items = df_items[['user_id','item_id','playtime_forever']]

In [40]:
df_items['playtime_forever'][df_items['playtime_forever']==0.0].value_counts()

playtime_forever
0.0    1787623
Name: count, dtype: int64

In [41]:
df_items.head()

,user_id,item_id,playtime_forever
0,76561197970982479,10,6.0
1,76561197970982479,20,0.0
2,76561197970982479,30,7.0
3,76561197970982479,40,0.0
4,76561197970982479,50,0.0


In [8]:
df_items.shape

(4182751, 3)

Remuevo los duplicados de df_items

In [42]:
df = df_items.drop_duplicates()

In [43]:
df.reset_index(inplace=True, drop=True)

In [44]:
df1 = df_steam_games.groupby('id').first().reset_index()

Ya que había pasado los nulos de 'anio' a 0, me quedo con todos los valores de anio que no sean 0.

In [45]:
df1 = df1[df1['anio']!=0]

Hago el merge (inner join) entre df_items y df_steam_games

In [46]:
resultado = df1.merge(df, left_on='id', right_on='item_id', how='inner')

In [47]:
resultado.drop('id',axis=1,inplace=True)

In [48]:
resultado.head()

,genres,anio,user_id,item_id,playtime_forever
0,action,2000,76561197970982479,10,6.0
1,action,2000,js41637,10,0.0
2,action,2000,riot punch,10,0.0
3,action,2000,doctr,10,93.0
4,action,2000,corruptedsoul,10,108.0


Estableciendo el género deseado para la prueba

In [66]:
genero = 'action'

Filtrando el dataframe resultado por el género deseado para la prueba.

In [67]:
df_filtrado = resultado[resultado['genres'] == genero]

Agrupando por año y sumando las horas jugadas por año.

In [68]:
tiempo_jugado = df_filtrado.groupby('anio')['playtime_forever'].sum()

Encontrando el usuario con más horas jugadas para ese género.

In [69]:
usuario_max = df_filtrado.groupby('user_id')['playtime_forever'].sum().idxmax()

Print del resultado

In [71]:
genre_playtime_dict = {
    "Usuario con más tiempo jugado": usuario_max,
    "Género": genero,
    "Tiempo jugado por año": tiempo_jugado.to_dict(),
    
}

print(genre_playtime_dict)

{'Usuario con más tiempo jugado': 'thugnificent', 'Género': 'action', 'Tiempo jugado por año': {1983: 3470.0, 1984: 384.0, 1988: 15582.0, 1989: 607.0, 1990: 15617.0, 1991: 2162.0, 1992: 1910.0, 1993: 17962.0, 1994: 78524.0, 1995: 163981.0, 1996: 52060.0, 1997: 249030.0, 1998: 1092993.0, 1999: 1065988.0, 2000: 986589.0, 2001: 691738.0, 2002: 355608.0, 2003: 2486614.0, 2004: 7989076.0, 2005: 3786045.0, 2006: 4679753.0, 2007: 8039524.0, 2008: 4916383.0, 2009: 15679430.0, 2010: 16435105.0, 2011: 33731428.0, 2012: 46919099.0, 2013: 42364566.0, 2014: 38149925.0, 2015: 29424070.0, 2016: 12842409.0, 2017: 11526961.0, 2018: 2224.0}}
